# Sophus tutorials
### Terminology
* SO3: rotation group in 3D
* SE3: rotation and translation group in 3D

### Supported features
#### SO3: Rotation group in 3D
* Initialize from rotational vector, quaternion, rotation matrix
* Convert to rotational vector, quaternion, rotation matrix
* Multiplication with SO3 or 3D points
* Operator [] for setting/getting items with index or slices
* Inverse, copy, print, and len
* Function vectorization

#### SE3
* Initialize from twist vector (rotation vector and translational vector), quaternion and translation, transformation matrix
* Convert to twist vector, quaternion and translation, transformation matrix
* Multiplication with SE3 or 3D points
* Operator [] for setting/getting items with index or slices
* Function vectorization
* Inverse, copy, print, and len
* Interpolate between two SE3
* Iterative mean of a group of SE3 


In [ ]:
from sophus_pybind import interpolate, iterativeMean, SE3, SO3
import numpy as np

# SO3: Rotation group in 3D

### From/to rotation vector
* rotation_vector: 3x1 or Nx3
* The magnitude of the vector is the rotation angle in rad, the rotation direction is the norm of the rotation_vector

In [ ]:
rotation_vector = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 1]])
so3vec_from_rotvec = SO3.exp(rotation_vector)
print(f"Initialize from rotation vector {rotation_vector}: \n resulting log: \n {so3vec_from_rotvec.log()}")

### From/to quaternion
* Quaternion in put is represented as w, xyz
  * Vectorized input is w_vec: Nx1 and xyz_vec: Nx3
* Quaternion is the **Hamilton** convention

In [ ]:
q_w = 1; q_xyz = np.array([0,0,0])
so3_from_quaternion = SO3.from_quat(q_w, q_xyz)
print(f"Initialize from quaternion [{q_w}, {q_xyz}]: \n resulting quaternion: \n {so3_from_quaternion.to_quat()}")

# Vectorized initialization
q_w_vec = np.array([1,0])
q_xyz_vec = np.array([[0,0,0], [0.0, 1.0, 0.0]])
so3vec_from_quaterion = SO3.from_quat(q_w_vec, q_xyz_vec)
for i in range(0,len(q_w_vec)):
    print(f"Initialize from quaternion [{q_w_vec[i]}, {q_xyz_vec[i]}]: \n resulting quaternion: \n{so3vec_from_quaterion.to_quat()[i]}")

### From/to rotation matrix

In [ ]:
rotation_matrix = [[-1,0,0], [0, 1,0 ], [0,0,-1]]
so3_from_matrix = SO3.from_matrix(rotation_matrix)
print(f"Initialize from matrix {rotation_matrix}: \n resulting matrix: \n{so3_from_matrix.to_matrix()}")

### Multiplication
* Matrix multiplication uses **@** operator
* Supports
  * Multiple SO3 @ Single SO3
  * Single SO3 @ Multiple SO3
  * Single SO3 @ Multiple 3D points

In [ ]:
so3_result_1 = so3_from_matrix @ so3vec_from_rotvec[0]
so3_result_2 = so3_from_matrix[0] @ so3vec_from_rotvec

original_points = np.random.rand(3,10)
points_from_a_single_so3 = so3vec_from_rotvec[0] @ original_points

### MISC
* Inverse
* copy, str, len, repr
* Operator [] to get/set from indices and slice

In [ ]:
so3_inverse = so3vec_from_rotvec.inverse()
so3_slice = so3_inverse[1:3]

import copy
print(f"A copy of the SO3 is {copy.copy(so3_inverse)}\n")
print(f"The str of the SO3 is {so3_inverse}\n")
print(f"The len of the SO3 is {len(so3_inverse)}\n")
print(f"The repr of the SO3 is {repr(so3_inverse)}\n")
print(f"A SO3 slice is {so3_slice}")

# SE3: transformation group in 3D

### From/to translational vector and rotation vector(twist)
* SE3.exp(translational_part, rotation_vector)
* [translational_part, rotation_vector] = SE3.log()
  
NOTE: translational_vector is not translation in SE3, it is off by a left jacobian matrix (ref: https://github.com/strasdat/Sophus/blob/main-1.x/sophus/se3.hpp#L859-L860)

In [ ]:
translational_part = [[1,2,3],[15,2,9],[10,4,3],[3,2,3]]
rotation_vector = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 1]]
se3_from_rotvec_and_translational = SE3.exp(translational_part, rotation_vector)
twist = se3_from_rotvec_and_translational.log()
print(f"Initialize from rotation vector {rotation_vector} and translational_part {translational_part}: \n \
resulting twist: \n{twist}")

### From/to quaternion and translation
* SE3.from_quat_and_translation(w, xyz, translation)
* [w, xyz, translation] = SE3.to_quat_and_translation()

In [ ]:
translation_vector = [[1,2,3],[15,2,9],[10,4,3],[3,2,3]]
w_vec = [1,0,0,1]
xyz_vec = [[0,0,0], [0,1,0], [0,0,1], [0,0,0]]
se3 = SE3.from_quat_and_translation(w_vec, xyz_vec, translation_vector)
quaternion_and_translation = se3.to_quat_and_translation()
print(f"Initialize from quaternion w {w_vec} and xyz {xyz_vec} \nwith translational_part {translational_part}: \n")
print(f"resulting quat and translation output : \n{quaternion_and_translation}")

### From/to transformation matrix of size 3x4 or 4x4
* SE3.from_matrix(matrix4x4)
* SE3.from_matrix3x4(matrix3x4)
* matrix4x4 = SE3.to_matrix()
* matrix3x4 = SE3.to_matrix3x4()

In [ ]:
transformation_matrix = np.array([[[ 1.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  2.],
        [ 0.,  0.,  1.,  3.],
        [ 0.,  0.,  0.,  1.]],

       [[-1.,  0.,  0., 15.],
        [ 0.,  1.,  0.,  2.],
        [ 0.,  0., -1.,  9.],
        [ 0.,  0.,  0.,  1.]],

       [[-1.,  0.,  0., 10.],
        [ 0., -1.,  0.,  4.],
        [ 0.,  0.,  1.,  3.],
        [ 0.,  0.,  0.,  1.]],

       [[ 1.,  0.,  0.,  3.],
        [ 0.,  1.,  0.,  2.],
        [ 0.,  0.,  1.,  3.],
        [ 0.,  0.,  0.,  1.]]])
se3_from_matrix = SE3.from_matrix(transformation_matrix)
se3_from_matrix3x4 = SE3.from_matrix3x4(transformation_matrix[:, 0:3,:])

### Multiplication
* Matrix multiplication uses **@** operator
* Supports
  * Multiple SE3 @ Single SE3
  * Single SE3 @ Multiple SE3
  * Single SE3 @ Multiple 3D points

In [ ]:
se3_result_1 = se3_from_matrix @ se3_from_rotvec_and_translational[0]
se3_result_2 = se3_from_matrix[0] @ se3_from_rotvec_and_translational

original_points = np.random.rand(3,10)
points_from_a_single_se3 = se3_from_matrix[0] @ original_points

### MISC
* Inverse
* copy, str, len, repr
* Operator [] to get/set from indices and slice

In [ ]:
se3_inverse = se3_from_matrix.inverse()
se3_slice = se3_inverse[0:2]

print(f"A copy of the SE3 is {copy.copy(se3_inverse)}\n")
print(f"The str of the SE3 is {se3_inverse}\n")
print(f"The len of the SE3 is {len(se3_inverse)}\n")
print(f"The repr of the SE3 is {repr(se3_inverse)}\n")
print(f"A SE3 slice is {se3_slice}")